# Data Cleaning and save Balance sheets

In [14]:
!pip install xlsxwriter

In [12]:
import os
import pandas as pd

# 🔧 Configuration
FOLDER_PATH = r"C:\Users\ASUS\Desktop\College_Last_sem_pro\git\Riskify\Data\raw\Company_Raw_Data\Cement"
SAVE_PATH = r"C:\Users\ASUS\Desktop\College_Last_sem_pro\College\Cleaned_Balance_Sheets"
TARGET_SHEETS = ["Balance Sheet", "Cash Flow", "Profit & Loss"]

# ✅ Create the output directory if it doesn't exist
os.makedirs(SAVE_PATH, exist_ok=True)

# 🧽 Enhanced cleaning function with label + index + type cleaning
def clean_df(df):
    df.columns = (
        df.columns
        .astype(str)
        .str.replace(r'\s+', ' ', regex=True)
        .str.replace(r'[^\w\s]', '', regex=True)
        .str.strip()
    )

    df.index = (
        df.index.astype(str)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
    )

    if df.shape[1] > 1:
        df.iloc[:, 0] = (
            df.iloc[:, 0].astype(str)
            .str.replace(r'\+', '', regex=True)
            .str.replace(r'%', '', regex=True)
            .str.replace(r'\s+', ' ', regex=True)
            .str.strip()
        )

    df = df.replace(r'%', '', regex=True)

    for col in df.columns[1:]:
        if not pd.api.types.is_numeric_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Drop the index entirely
    df.reset_index(drop=True, inplace=True)

    df.rename(columns={'Unnamed 0': 'Financial_Metric'}, inplace=True)

    df = df.dropna(thresh=0.5 * len(df), axis=1)

    df.set_index(df.columns[0], inplace=True)

    return df


# 📂 Process files and clean Balance Sheet sheets only
def load_clean_and_save_balance_sheets(folder_path, target_sheets):
    cleaned = {}

    for file in os.listdir(folder_path):
        if file.endswith(('.xlsx', '.xls')) and not file.startswith('~$'):
            file_path = os.path.join(folder_path, file)
            company = os.path.splitext(file)[0]
            try:
                xls = pd.ExcelFile(file_path)
                for sheet in target_sheets:
                    if sheet == 'Balance Sheet' and sheet in xls.sheet_names:
                        df = pd.read_excel(xls, sheet_name=sheet)
                        df = clean_df(df)

                        # Identify year columns
                        year_cols = [col for col in df.columns if str(col).isdigit()]

                        # Interpolation using polynomial method
                        numeric_data = df[year_cols].copy()
                        numeric_data.columns = numeric_data.columns.astype(int)
                        numeric_data = numeric_data.transpose()
                        numeric_data.index = numeric_data.index.astype(int)

                        interpolated = numeric_data.interpolate(method='polynomial', order=2, limit_direction='both').transpose()
                        df[year_cols] = interpolated

                        cleaned[f"{company}_{sheet}"] = df

                        # ✅ Save to CSV
                        save_filename = f"{company}_balance_sheet.csv"
                        df.to_csv(os.path.join(SAVE_PATH, save_filename))
                        print(f"✅ Saved: {save_filename}")

            except Exception as e:
                print(f"❌ Error processing {file}: {e}")

    return cleaned

# ✅ Main
if __name__ == "__main__":
    cleaned_balance_sheets = load_clean_and_save_balance_sheets(FOLDER_PATH, TARGET_SHEETS)


✅ Saved: ACC_balance_sheet.csv
✅ Saved: AMBUJACEM_balance_sheet.csv
✅ Saved: DALBHARAT_balance_sheet.csv
✅ Saved: SHREECEM_balance_sheet.csv
✅ Saved: ULTRACEMCO_balance_sheet.csv


# Combining BalanceSheets of all Companies in of single sector

In [18]:
import os
import pandas as pd
import re

# 🔧 Configuration
FOLDER_PATH = r"C:\Users\ASUS\Desktop\College_Last_sem_pro\College\Cleaned_Balance_Sheets"
OUTPUT_FILE = r"C:\Users\ASUS\Desktop\College_Last_sem_pro\College\Combined_Balance_Sheet_Averaged.xlsx"
METRIC_COLUMN_NAME = "Financial_Metric"
SHEET_NAME = "All_Companies"

# 📂 Function to combine and average month-wise columns into year columns
def combine_csvs_to_excel(folder_path, output_file, metric_column=METRIC_COLUMN_NAME, sheet_name=SHEET_NAME):
    all_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            file_path = os.path.join(folder_path, filename)
            company_name = os.path.splitext(filename)[0]

            try:
                df = pd.read_csv(file_path)

                # ➕ Insert 'Company' column
                if metric_column in df.columns:
                    col_list = df.columns.tolist()
                    insert_idx = col_list.index(metric_column) + 1
                    df.insert(insert_idx, 'Company', company_name)
                else:
                    df['Company'] = company_name

                # ✅ Step 1: Extract year from each column and group
                fixed_cols = [metric_column, 'Company']
                data_cols = [col for col in df.columns if col not in fixed_cols]

                # Mapping: year → [columns with that year]
                year_col_map = {}
                for col in data_cols:
                    match = re.search(r'(\d{4})', col)
                    if match:
                        year = int(match.group(1))  # convert to int
                        year_col_map.setdefault(year, []).append(col)

                # ✅ Step 2: Collapse columns of same year by averaging
                collapsed_df = df[fixed_cols].copy()
                for year, cols in year_col_map.items():
                    year_values = df[cols].apply(pd.to_numeric, errors='coerce')
                    collapsed_df[year] = year_values.mean(axis=1)

                all_data.append(collapsed_df)

            except Exception as e:
                print(f"❌ Error processing {filename}: {e}")

    # ✅ Step 3: Concatenate and export
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)

        # Ensure all year columns are numeric and sorted
        fixed_cols = [metric_column, 'Company']
        year_cols = [col for col in combined_df.columns if col not in fixed_cols]
        numeric_year_cols = sorted([int(col) for col in year_cols])
        final_df = combined_df[fixed_cols + numeric_year_cols]

        with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
            final_df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"✅ Combined and averaged Excel saved: {output_file}")
    else:
        print("⚠️ No valid CSV files found.")

# 🚀 Main
if __name__ == "__main__":
    combine_csvs_to_excel(FOLDER_PATH, OUTPUT_FILE)


✅ Combined and averaged Excel saved: C:\Users\ASUS\Desktop\College_Last_sem_pro\College\Combined_Balance_Sheet_Averaged.xlsx
